In [1]:
import numpy as np
import h5py
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import yaml
import argparse
import scipy.optimize

sys.path.append('../')
import utils
import settings
plt.style.use('../spectrum.mplstyle')

In [2]:
n=10
all_data = {
    "Hg(0) no D": {
        "levels": "Hg(0)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_Hg_P0_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_Hg_P0_noD_single_pivot_n8_m8_d16_c50.csv",
    },
    "G1(1) no D": {
        "levels": "G1(1)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_G1_P1_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_G1_P1_noD_single_pivot_n8_m8_d16_c50.csv",
    },
    "G2(1) no D": {
        "levels": "G2(1)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_G2_P1_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_G2_P1_noD_single_pivot_n8_m8_d16_c50.csv",
    },
    "G(3) no D": {
        "levels": "G(3)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_G_P2_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_G_P2_noD_single_pivot_n8_m8_d16_c50.csv",
    },
#     "F1(3) no D": {
#         "levels": "F2(3)_levels.csv",
#         "reduced_levels": "withoutD\gevp_isoquartet_S0_F1_P3_noD_single_pivot_n8_m8_d16_c50.csv",
#         "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_F1_P3_noD_single_pivot_n8_m8_d16_c50.csv",
#     },
    "F2(3) no D": {
        "levels": "F2(3)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_F2_P3_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_F2_P3_noD_single_pivot_n8_m8_d16_c50.csv",
    },
    "G(3) no D": {
        "levels": "G(3)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_G_P3_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_G_P3_noD_single_pivot_n8_m8_d16_c50.csv",
    },
    "G1(4) no D": {
        "levels": "G1(4)_levels.csv",
        "reduced_levels": "withoutD\gevp_isoquartet_S0_G1_P4_noD_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_G1_P4_noD_single_pivot_n8_m8_d16_c50.csv",
    },
#     "G2(4) no D": {
#         "levels": "G1(4)_levels.csv",
#         "reduced_levels": "withoutD\gevp_isoquartet_S0_G2_P4_noD_single_pivot_n8_m8_d16_c50.csv",
#         "level_inserts": "withoutD\gevp_inserts_isoquartet_S0_G2_P4_noD_single_pivot_n8_m8_d16_c50.csv",
#     },
    "Hg(0) D": {
        "levels": "Hg(0)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_Hg_P0_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_Hg_P0_single_pivot_n8_m8_d16_c50.csv",
    },
    "G1(1) D": {
        "levels": "G1(1)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_G1_P1_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_G1_P1_single_pivot_n8_m8_d16_c50.csv",
    },
    "G2(1) D": {
        "levels": "G2(1)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_G2_P1_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_G2_P1_single_pivot_n8_m8_d16_c50.csv",
    },
    "G(3) D": {
        "levels": "G(3)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_G_P2_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_G_P2_single_pivot_n8_m8_d16_c50.csv",
    },
    "F1(3) D": {
        "levels": "F2(3)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_F1_P3_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_F1_P3_single_pivot_n8_m8_d16_c50.csv",
    },
    "F2(3) D": {
        "levels": "F2(3)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_F2_P3_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_F2_P3_single_pivot_n8_m8_d16_c50.csv",
    },
    "G(3) D": {
        "levels": "G(3)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_G_P3_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_G_P3_single_pivot_n8_m8_d16_c50.csv",
    },
    "G1(4) D": {
        "levels": "G1(4)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_G1_P4_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_G1_P4_single_pivot_n8_m8_d16_c50.csv",
    },
    "G2(4) D": {
        "levels": "G1(4)_levels.csv",
        "reduced_levels": "withD\gevp_isoquartet_S0_G2_P4_single_pivot_n8_m8_d16_c50.csv",
        "level_inserts": "withD\gevp_inserts_isoquartet_S0_G2_P4_single_pivot_n8_m8_d16_c50.csv",
    },
    
}

In [ ]:
# i=0

f = plt.figure()

for plots in all_data:
    
    min_level = 500
    max_level = 0.0
    level_inserts = pd.read_csv(all_data[plots]["level_inserts"])
    n_plots = int(max(level_inserts['corr'])+1)
    f, axes = plt.subplots(n_plots, 1)
    f.set_figwidth(10)
    f.set_figheight(6*n_plots)

    full_spec = pd.read_csv(all_data[plots]["levels"])
    levels = pd.read_csv(all_data[plots]["reduced_levels"])

    x=[0.0]*n_plots
    total_shift = [0.0]*n_plots
    d=0.3
    for index,row in level_inserts.iterrows():
        ic = int(row['corr'])
        if row['prior']:
            mfc = settings.colors[ic]
            axes[ic].errorbar(x=x[ic]+d,y=row['energy'],yerr=row['energyerr'], linewidth=0.0, capsize=5, elinewidth=1.5,color=settings.colors[ic], marker=settings.markers[ic],mfc=mfc)
            axes[ic].plot(x[ic],row['insert'], linewidth=0.0,color=settings.colors[ic], marker=settings.markers[ic],mfc=mfc)
            axes[ic].arrow(x[ic],row['insert'],d,row['energy']-row['insert'],width=0.00025,zorder=4,ec=None, color="black")
            x[ic]+=2*d
            total_shift[ic] += (row['energy']-row['insert'])**2/row['energyerr']/row['energyerr']
        else:
            mfc = "white"
        
    for i in range(n_plots):
        if x[i]:    
            xall = [x[i]+d]*len(full_spec['energy'])
            axes[i].errorbar(x=xall,y=full_spec['energy'],yerr=full_spec['energyerr'], linewidth=0.0, capsize=5, elinewidth=1.5,color=settings.colors[n_plots], marker=settings.markers[n_plots])


    for i in range(n_plots):
        xrange = axes[i].get_xlim()
        for val,err in zip(levels['energy'],levels['energyerr']):
            axes[i].hlines(val,xrange[0],xrange[1],color="black")
            axes[i].fill_between(xrange,val-err,val+err,color="gray",zorder=0)
            if val<min_level:
                min_level=val
            if val>max_level:
                max_level=val

        dE = (max_level-min_level)/(n+1)
        for j in range(n):
            axes[i].hlines(min_level+dE*(j+1),xrange[0],xrange[1],color="black",ls="--", zorder = 3)
        axes[i].set_xticks([])
        if total_shift[i]:
            axes[i].text(0.2,max_level,total_shift[i])


    axes[0].set_title(plots)
    plt.tight_layout()
    plt.savefig(f"{plots}.png")